In [2]:
! pip install .\dist\quick_sentiments-0.1.8-py3-none-any.whl

# the package can be seperately installed from PyPI however the dependencies are not up to date on PyPI
# pip install -i https://test.pypi.org/simple/ quick-sentiments 


Processing c:\users\meala\documents\quick_sentiments\dist\quick_sentiments-0.1.8-py3-none-any.whl
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.7.1-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached polars-0.19.19-cp38-abi3-win_amd64.whl.metadata (15 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached xgboost-3.0.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached scipy-1.16.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached regex-2024.11.6-cp311-cp311-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached smart_open-7.3.0.post1-py3-none-any.whl.metadata (24 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cache

In [ ]:
import polars as pl
#make sure that this is the main file
import sys
import os
project_root = os.getcwd()
sys.path.insert(0, project_root)

# here I have three python script I built to pre_process the data and running the pipeline
# you can find the code in the tools/preprocess.py file
# you can find  the code in the tools/pipeline.py file
# the pre_process function is used to clean the text data, there are various options available, please check the tools/preprocess.py file for details
# the run_pipeline function is used to run the sentimental analysis pipeline, it takes the training data and the vectorizer and machine learning methods as input, and returns the results
from quick_sentiments import pre_process
from quick_sentiments import run_pipeline
from quick_sentiments import make_predictions

### Training Dataset


Name your data as Train.csv and place it in the Training Data folder. Or you can change the path in the code below.


In [5]:
# keep you training dataset in the training data folder
# this template uses csv files 
# column names can be set in Python but this template does not automatically update the column for the demo 
# however, the function will give you the option to tell column names for the text and label data

df_train = pl.read_csv("demo/training_data/train.csv",encoding='ISO-8859-1') 
print(f"Dataset shape: {df_train.shape[0]} rows and {df_train.shape[1]} columns")


Dataset shape: 162758 rows and 5 columns


### DEMO

In [6]:
df_train.head()
# randomly select only 10% of the data since the dataset is large
#RUN ONLY ONCE
df_train = df_train.sample(fraction=0.1, shuffle=True, seed=42) 
df_train

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
str,str,bool,str,str
"""john_mcclane_r…","""Deborah Farley…",false,"""..a cocksure, …","""POSITIVE"""
"""tyler_durden_d…","""Margaret Marti…",false,"""A wild ride of…","""POSITIVE"""
"""glimmer_mythic…","""Angel Peters""",false,"""In a year of m…","""POSITIVE"""
"""gandalf_journe…","""Alexandria Wil…",true,"""Festival will …","""POSITIVE"""
"""vito_corleone_…","""Andrew Blanken…",false,"""The limp&#44; …","""NEGATIVE"""
"""zephyr_scarlet…","""Lee Griffin""",false,"""The film does …","""POSITIVE"""
"""the_joker_ques…","""Brianna Flores…",true,"""A good-hearted…","""POSITIVE"""
"""surreal_the_ca…","""Shelia Miller""",false,"""At this moment…","""POSITIVE"""
"""dorothy_gale_g…","""Chelsea Martin…",false,"""Beats thrums w…","""POSITIVE"""


The dataset is for training. The sentiments are already labeled. This will allow us to train a model that can predict sentiments on new data.


In [7]:
# you can use the pre_process function to clean the text data
response_column = "reviewText" # this is the column name for the text data, feel free to change it to your text column name
sentiment_column = "sentiment" # this is the column name for the sentiment data, feel free to change it to your sentiment column name
print(df_train[response_column][2])
print("\n" + pre_process(df_train[response_column][2]) )

In a year of many car movies, this one was the best (Living Life Fearless)

year many car movie one best living life fearless


In [8]:
# make changes as necessary
# inside the map_elements, add  the parameters [pre_process(x, parameters_to_be_added)] and set it True/False if it differs from the defualt value
# check the tools/preprocess.py file for the parameters and their default values
# some of the parameters are remove_brackets, remove_stopwords, remove_punctuation, remove_numbers, remove_emojis, remove_urls, remove_html_tags, lemmatize, stem, lowercase
df_train = df_train.with_columns(
    pl.col(response_column).map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
df_train.head(10)

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,processed
str,str,bool,str,str,str
"""john_mcclane_r…","""Deborah Farley…",false,"""..a cocksure, …","""POSITIVE""","""cocksure styli…"
"""tyler_durden_d…","""Margaret Marti…",false,"""A wild ride of…","""POSITIVE""","""wild ride movi…"
"""glimmer_mythic…","""Angel Peters""",false,"""In a year of m…","""POSITIVE""","""year many car …"
"""gandalf_journe…","""Alexandria Wil…",true,"""Festival will …","""POSITIVE""","""festival surel…"
"""vito_corleone_…","""Andrew Blanken…",false,"""The limp&#44; …","""NEGATIVE""","""limp 44 neglig…"
"""zephyr_scarlet…","""Lee Griffin""",false,"""The film does …","""POSITIVE""","""film weak spot…"
"""the_joker_ques…","""Brianna Flores…",true,"""A good-hearted…","""POSITIVE""","""goodhearted en…"
"""surreal_the_ca…","""Shelia Miller""",false,"""At this moment…","""POSITIVE""","""moment 500 day…"
"""dorothy_gale_g…","""Chelsea Martin…",false,"""Beats thrums w…","""POSITIVE""","""beat thrum unb…"


In [ ]:
#### in this template, there are four text representation / vectorizer methods available 
#### in the function run_pipeline (in python cell below), we shall make use of this, write the words inside [ ] for the methods you want to use
#### 1. Bag of Words [BOW] 
#### 2. Term Frequency [tf]
#### 3. TF -IDF    [tfidf]
#### 4. Word Embedding using Word2Vec (you can use other packages with slight changes) [wv] 
         # Word Embedding uses defualt 300 values; this will take some time to run

In [ ]:
#### in this template, there are also three machine learning methods that can be used
#### 1. Logistic Regression [logit]
#### 2. Random forest (recommended) (rf)
#### 3. XGBoosting  [XGB](word embedding and XGBoost may take long time to complete, combination of both is not recommended in local machine)

#I will keep this repository updated, and I will add more methods in the future

In [9]:
# this is the example of how to use the function
# you can change the vectorizer_name and model_name to the ones you want to use
# for now we will use word embedding and logistic regression
# write the name of your columns in the text_column_name and sentiment_column_name
# the text_column_name is the column name of the text data, and sentiment_column_name is

# run_pipeline function will return the dataframe with the vectorized text, vectorizer used  and the model
# it will also print the results of the model, including the accuracy and F1 score
# note, even without hyperparameter tuning, the model is getting over 70% accuracy in my test
# there may not be a need to perform hyperparameter tuning, but you can set perform_tuning to True if you want to do that

dt= run_pipeline(
    vectorizer_name="wv", # BOW, tf, tfidf, wv
    model_name="logit", # logit, rf, XGB .#XGB takes long time, can not recommend using it on normal case
    df=df_train,
    text_column_name="processed",  # this is the column name of the text data, 
    sentiment_column_name = "sentiment",
    perform_tuning = False # make this true if you want to perform hyperparameter tuning, it will take longer time and 
                            # may run out of memory if the dataset is large,
)

# missing values in the text data will be removed

--- Running Pipeline for Wv + Logit ---
Error loading vectorizer module/function: cannot import name 'triu' from 'scipy.linalg' (c:\Users\meala\anaconda3\envs\for_quick_sentiments\Lib\site-packages\scipy\linalg\__init__.py)


In [7]:
## the dt is a dictionary that contains the results of the model, including the accuracy and F1 score
print(dt.keys())
# you can access the results using the keys of the dictionary
print("Vectorizer used: ", dt["vectorizer_name"])
print("Model used: ", dt["model_object"])
print("Accuracy: ", dt["accuracy"])



dict_keys(['model_object', 'vectorizer_name', 'vectorizer_object', 'label_encoder', 'y_test', 'y_pred', 'accuracy', 'report'])
Vectorizer used:  BOW
Model used:  LogisticRegression(max_iter=500, random_state=42, solver='liblinear')
Accuracy:  0.7536


### New Dataset for prediction
You can use the same format as the training dataset, but ensure that it contains the "Response" column for text data. The "Sentiment" column is optional for prediction datasets, as it will be generated by the model.
Make sure the dataset is saved in the "New Data" folder and is in CSV format.

In [8]:
new_data = pl.read_csv("demo/new_data/test.csv",encoding='ISO-8859-1') #keep your file here
print(new_data.shape)
new_data= new_data.sample(fraction=0.25, shuffle=True, seed=42)
print(new_data.shape)

(55315, 4)
(13828, 4)


In [9]:
new_data = new_data.with_columns(
    pl.col(response_column).map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
new_data.head(10)

movieid,reviewerName,isTopCritic,reviewText,processed
str,str,bool,str,str
"""frodo_baggins_…","""Toni Vaughn""",false,"""The result is …","""result unsettl…"
"""stardust_john_…","""Carol Jennings…",false,"""Think twice ab…","""think twice ge…"
"""hermione_grang…","""Tara Huang""",true,"""A film that's …","""film bloody wo…"
"""astonish_valia…","""Shelley Murill…",false,"""...a decent se…","""decent setup e…"
"""indiana_jones_…","""Daniel Bond""",false,"""Inspiring? Not…","""inspiring lame…"
"""glimmer_hannib…","""Mrs. Nicole Fl…",false,"""This perfectly…","""perfectly exec…"
"""brave_anakin_s…","""Melissa Harrin…",false,"""The Richard Cu…","""richard curtis…"
"""katniss_everde…","""Mckenzie Ortiz…",false,"""A fun-filled a…","""funfilled afte…"
"""secret_magic_j…","""Samantha Ware""",false,"""I'm still not …","""still sure sup…"


In [10]:
make_predictions(
    new_data=new_data,
    text_column_name="processed",
    vectorizer=dt["vectorizer_object"],
    best_model=dt["model_object"],
    label_encoder=dt["label_encoder"],
    prediction_column_name="sentiment_predictions"  # Optional custom name
)

movieid,reviewerName,isTopCritic,reviewText,processed,sentiment_predictions
str,str,bool,str,str,str
"""frodo_baggins_…","""Toni Vaughn""",false,"""The result is …","""result unsettl…","""POSITIVE"""
"""stardust_john_…","""Carol Jennings…",false,"""Think twice ab…","""think twice ge…","""NEGATIVE"""
"""hermione_grang…","""Tara Huang""",true,"""A film that's …","""film bloody wo…","""POSITIVE"""
"""astonish_valia…","""Shelley Murill…",false,"""...a decent se…","""decent setup e…","""NEGATIVE"""
"""indiana_jones_…","""Daniel Bond""",false,"""Inspiring? Not…","""inspiring lame…","""NEGATIVE"""
"""glimmer_hannib…","""Mrs. Nicole Fl…",false,"""This perfectly…","""perfectly exec…","""POSITIVE"""
"""brave_anakin_s…","""Melissa Harrin…",false,"""The Richard Cu…","""richard curtis…","""POSITIVE"""
"""katniss_everde…","""Mckenzie Ortiz…",false,"""A fun-filled a…","""funfilled afte…","""NEGATIVE"""
"""secret_magic_j…","""Samantha Ware""",false,"""I'm still not …","""still sure sup…","""NEGATIVE"""
